In [4]:
from bs4 import BeautifulSoup
import requests
import json

In [5]:
Website_name='Camp Quest'
Website_url='https://campquest.org/'
HEADERS =({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
                   'Accept-Language':'en-US, en;q=0.5'})

home = requests.get(Website_url,headers=HEADERS)
home_soup = BeautifulSoup(home.content, 'lxml')

In [6]:
image_url = []
img_table = home_soup.find_all('div', class_='elementor-widget-container')

for img in img_table:
    img_tag = img.find('img')
    if img_tag:
        url = img_tag.get('src')
        if url:
            image_url.append(url)

image_url.pop()

image_url


['https://campquest.org/wp-content/uploads/2024/12/10k-donation-match-12-20-24-1024x538.png',
 'https://campquest.org/wp-content/uploads/2021/04/40253695_2249546175074369_6659048747487461376_o.jpg',
 'https://campquest.org/wp-content/uploads/2021/06/Copy-of-IMG_0044-1024x768.jpg']

In [7]:
desc1_table=home_soup.find_all('div',attrs={'class':'elementor-widget-container'})
description=desc1_table[4].text.strip()

In [8]:
# Provided data
Location = 'PO Box 449 Pickens, SC'
Telephone = '(540) 209-7978'
Mail = 'camp@campquest.org'

zip_code='29671'

latitude = '34.93492'  # Placeholder value, replace with actual latitude
longitude = '-82.71331'  # Placeholder value, replace with actual longitude

# Create the location_data and contact_data dictionaries
location_data = {
    "address": Location,
    "zip_code": zip_code,
    "latitude": latitude,
    "longitude": longitude
}

contact_data = {
    "mail": Mail,
    "telephone": Telephone
}

# Output the results
print(location_data)
print(contact_data)


{'address': 'PO Box 449 Pickens, SC', 'zip_code': '29671', 'latitude': '34.93492', 'longitude': '-82.71331'}
{'mail': 'camp@campquest.org', 'telephone': '(540) 209-7978'}


In [9]:
logo_tag = home_soup.find('a', class_='custom-logo-link').find('img')
logo_url = logo_tag['src']
print(logo_url)

https://campquest.org/wp-content/uploads/2024/12/Camp-Quest-logo-2024-square-white-1-120x120.png


In [10]:
Socials=[]
Facebook = home_soup.find('a', attrs={'class': 'ast-builder-social-element ast-inline-flex ast-facebook footer-social-item'}).get('href')
Socials.append({'facebook': Facebook})

# Add Instagram link
Instagram = home_soup.find('a', attrs={'class': 'ast-builder-social-element ast-inline-flex ast-instagram footer-social-item'}).get('href')
Socials.append({'instagram': Instagram})

# Add Tiktok link
twitter = home_soup.find('a', attrs={'class': 'ast-builder-social-element ast-inline-flex ast-twitter footer-social-item'}).get('href')
Socials.append({'twitter': twitter})
Socials

[{'facebook': 'https://www.facebook.com/CampQuest/'},
 {'instagram': 'https://www.instagram.com/campquest/'},
 {'twitter': 'https://twitter.com/CampQuest'}]

In [11]:
rd_url = 'https://campquestnorth.org/registerandaboutcamp/'

rd = requests.get(rd_url, headers=HEADERS)
rd_soup = BeautifulSoup(rd.content, 'lxml')

# Debugging: Save the HTML to analyze structure
with open("debug.html", "w", encoding="utf-8") as f:
    f.write(rd_soup.prettify())

# Locate the "2024 Dates" section
dates_section = None
for h3 in rd_soup.find_all('h3'):
    if "2024 Dates" in h3.get_text():
        dates_section = h3.find_next('p')
        break

if not dates_section:
    print("Could not find the '2024 Dates' section.")
    exit()

# Extract details
camp_details = [
    str(detail).strip() for detail in dates_section.contents if isinstance(detail, str) and detail.strip()
]

# Define a helper function to parse each camp's details
def parse_camp_detail(detail):
    try:
        parts = detail.split(":")
        name = parts[0].strip()
        rest = parts[1].strip().split("(")
        date_range, location = rest[0].strip(), rest[1].replace(")", "").strip()
        start_date, end_date = date_range.split("-")
        return {
            "name": name,
            "start_date": start_date.strip(),
            "end_date": end_date.strip(),
            "location": location,
            "cost": "$735",
            "eligible": "8-17",
            'duration':'null',
            'deposits':'null'
        }
    except Exception as e:
        print(f"Error parsing detail: {detail}")
        raise e

# Process each camp detail
camps = [parse_camp_detail(detail) for detail in camp_details]

camps

[{'name': 'Minnesota Camp Week 1',
  'start_date': 'July 14th',
  'end_date': '20th',
  'location': 'Mound, MN',
  'cost': '$735',
  'eligible': '8-17',
  'duration': 'null',
  'deposits': 'null'},
 {'name': 'Minnesota Camp Week 2',
  'start_date': 'July 21st',
  'end_date': '27th',
  'location': 'Mound, MN',
  'cost': '$735',
  'eligible': '8-17',
  'duration': 'null',
  'deposits': 'null'},
 {'name': 'Iowa Camp',
  'start_date': 'August 4th',
  'end_date': '10th',
  'location': 'Boone, IA',
  'cost': '$735',
  'eligible': '8-17',
  'duration': 'null',
  'deposits': 'null'}]

In [12]:
cat_url = 'https://campquest.org/camps/programs/'
cat = requests.get(cat_url, headers=HEADERS)
cat_soup = BeautifulSoup(cat.content, 'lxml')

categories = []
cat_all = cat_soup.find_all('div', class_='elementor-widget-wrap elementor-element-populated')

for i in cat_all[2:6]:
    cat_name = i.find('strong')
    cat_desc = i.find('p')
    cat_image = i.find('img')
    
    if cat_name and cat_desc:
        name = cat_name.text.strip()
        description = cat_desc.text.strip().replace('\u201c','').replace('\u201d','').replace('\u2014','').replace('\u2019','').replace('\u2026','').replace('\u00c9','').replace('\u2122','')
        image_url = cat_image['src']
        
        if description.startswith(name):
            description = description[len(name):].strip()
        
        categories.append({
            'name': name,
            'description': description,
            'image': image_url
        })


In [13]:
cat_url = 'https://campquest.org/camps/programs/'
cat = requests.get(cat_url, headers=HEADERS)
cat_soup = BeautifulSoup(cat.content, 'lxml')

activities = []
act_all = cat_soup.find_all('div', class_='elementor-widget-wrap elementor-element-populated')

for i in act_all[6:12]:
    act_name = i.find('strong')
    act_desc = i.find('p')  
    act_image = i.find('img')
    
    if cat_name and cat_desc:
        aname = act_name.text.strip().replace('\u00c9','')
        adescription = act_desc.text.strip().replace('\u201c','').replace('\u201d','').replace('\u2014','').replace('\u2019','').replace('\u2026','').replace('\u00c9','').replace('\u2122','')
        aimage_url = act_image['src']
        if adescription.startswith(aname):
            adescription = adescription[len(aname):].strip()
        activities.append({
            'name': aname,
            'description': adescription,
            'image': aimage_url
        })


In [14]:
pr_url = 'https://campquest.org/privacy-policy/'
pr = requests.get(pr_url, headers=HEADERS)
pr_soup = BeautifulSoup(pr.content, 'lxml')
pr_soup

# Find all the section headings (e.g., <h2> tags)
privacy_head = pr_soup.find_all('h2', class_='elementor-heading-title elementor-size-default')

privacy_name=[]

for i in privacy_head:
    p_tag = i.find('span')
    if p_tag:
        privacy_name.append({'name': p_tag.text.strip()})
privacy_name

[{'name': 'The Information We Collect'},
 {'name': 'How We Use This Information'},
 {'name': 'Who We Share This Information With'},
 {'name': 'Security'},
 {'name': 'Accessing Information'}]

In [15]:
# Find all the sections with the descriptions (e.g., <div> elements containing the descriptions)
privacy_all = pr_soup.find_all('div', class_='elementor-widget-container')
privacy_all

# List to store the names and descriptions
privacy_data = []

# Extract the descriptions (from <p> tags within the found sections)
for i in privacy_all:
    p_tag = i.find('p')
    if p_tag:
        privacy_data.append({'description': p_tag.text.strip()})

privacy_data

[{'description': 'Camp Quest collects information by various methods including information actively provided by its lead providers and customers, as well as information arising from customer surveys and general feedback. The types of personal information we collect include name, address, phone number, and email address.'},
 {'description': 'This information is used to aid in the provision of our various products and services, including [customer service, accounting, billing, collections, and the marketing of other products and services]. Camp Quest may use aggregate or anonymous information for various uses for itself and third parties.'},
 {'description': 'Camp Quest does not share personal information with any third parties except as disclosed in this policy. Camp Quest may provide personal information to subcontractors and professional advisers (which shall be bound by privacy obligations) to assist Camp Quest’s uses disclosed herein.'},
 {'description': 'Personal information is sto

In [16]:
# Combining names and descriptions into a single list
privacy_combined = []

# Ensure that both lists have the same length
for name, data in zip(privacy_name, privacy_data):
    privacy_combined.append({
        'name': name['name'],
        'description': data['description']
    })

privacy_combined


[{'name': 'The Information We Collect',
  'description': 'Camp Quest collects information by various methods including information actively provided by its lead providers and customers, as well as information arising from customer surveys and general feedback. The types of personal information we collect include name, address, phone number, and email address.'},
 {'name': 'How We Use This Information',
  'description': 'This information is used to aid in the provision of our various products and services, including [customer service, accounting, billing, collections, and the marketing of other products and services]. Camp Quest may use aggregate or anonymous information for various uses for itself and third parties.'},
 {'name': 'Who We Share This Information With',
  'description': 'Camp Quest does not share personal information with any third parties except as disclosed in this policy. Camp Quest may provide personal information to subcontractors and professional advisers (which shal

In [17]:
Camp_Quest= [
    {'name': Website_name,  
    'url': Website_url,
    'logo': logo_url,
    'description':description,
    'location': location_data,
    'contact': contact_data,
    'socials': Socials,
    'image_urls':image_url,
    'sessions':camps,
    'categories': categories,
    'activities': activities,
    'policies': privacy_combined}
]
file_path = 'Camp_Quest.json'

with open(file_path, 'w') as json_file:
    json.dump(Camp_Quest, json_file, indent=4)